In [1]:
import random
import pandas as pd
import numpy as np
# import datetime

from copy import deepcopy

from pathlib import Path
import sys
import importlib

from easydict import EasyDict
from pprint import pprint
import torch
import os
import time

import time
from tqdm import tqdm
import datetime

In [2]:
BASE_DIR = Path('../Pointnet_Pointnet2_pytorch')
sys.path.append(str(BASE_DIR))
sys.path.append(str(BASE_DIR/'models'))
print(sys.path)

from data_utils.ModelNetDataLoader import ModelNetDataLoader
from pointnet2_utils import farthest_point_sample

RESULT_DIR = BASE_DIR/'../Mywork/Results'
FIGURE_DIR = RESULT_DIR/'Figures'
FILE_DIR = RESULT_DIR/'Files'

['/mnt/workspace/Github/Share/Mywork', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/tmp/OpenPCDet', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/root/.ipython', '../Pointnet_Pointnet2_pytorch', '../Pointnet_Pointnet2_pytorch/models']


In [3]:
#GA_downsampling用のargs
args = EasyDict({
    'use_cpu': 'False',
    'gpu': '0',
    'batch_size': 103, 
    'num_category': 40,
    'num_point': 1024,
    'log_dir': 'pointnet2_cls_ssg',
    'use_normals': False, #bestを使うときはTrue
    'use_uniform_sample': False, #Trueにするとデータロード時にFPS実行。FPSには初期値にランダム性あり。TrueとFalseが逆な気がする
    'num_votes': 3,
    'output': FILE_DIR,
    })

num_class = args.num_category
print("The num of class is {0}".format(num_class))
print(args.log_dir)

The num of class is 40
pointnet2_cls_ssg


In [5]:
data_path = BASE_DIR / Path('data/modelnet40_normal_resampled/')

dataset = ModelNetDataLoader(root=data_path, args=args, split='test', process_data=False)
DataLoader = torch.utils.data.DataLoader(dataset, batch_size=args.batch_size, shuffle=False, num_workers=0)

The size of test data is 2468


In [6]:
model_name = "cls_ssg_feature_selection" #feature_selectionっていう名前だけど、実態は２つに分けたもの
model = importlib.import_module(model_name)

Feature_vector_calculator = model.get_first_model(num_class, normal_channel=args.use_normals)
Feature_vector_calculator = Feature_vector_calculator.cuda()
Classifier_from_feature_vector = model.get_second_model(num_class, normal_channel=args.use_normals)

experiment_dir = BASE_DIR / Path('log/classification/') / Path(args.log_dir)
checkpoint = torch.load(str(experiment_dir) + '/checkpoints/best_model.pth') #学習済みデータを使う

Feature_vector_calculator.load_state_dict(checkpoint['model_state_dict'])
Classifier_from_feature_vector.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [7]:
def initialize(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [10]:
iteration = 10
feature_vector = torch.empty(iteration, len(DataLoader.dataset), 1024)
seed = 0
initialize(seed)

for j in range(iteration):
    with torch.no_grad():
        start_time = time.perf_counter()
        for i, (points, _) in tqdm(enumerate(DataLoader), total=len(DataLoader)):
            B, _, _ = points.shape
            points = points.cuda()
            points = points.transpose(2, 1)
            Feature_vector_calculator = Feature_vector_calculator.eval().cuda()

            feature_vector[j, i * args.batch_size:i * args.batch_size + B] = Feature_vector_calculator(points)
        stop_time = time.perf_counter()
        elapsed_time = stop_time - start_time
    
    print(elapsed_time)

100%|██████████████████████████████████████████████████████████████████████████████████████| 24/24 [03:40<00:00,  9.20s/it]


220.69828344741836


100%|██████████████████████████████████████████████████████████████████████████████████████| 24/24 [03:22<00:00,  8.42s/it]


202.17815473629162


100%|██████████████████████████████████████████████████████████████████████████████████████| 24/24 [03:23<00:00,  8.48s/it]


...

202.17764853918925


100%|██████████████████████████████████████████████████████████████████████████████████████| 24/24 [03:22<00:00,  8.43s/it]


202.23140434967354


100%|██████████████████████████████████████████████████████████████████████████████████████| 24/24 [03:22<00:00,  8.44s/it]

202.66234683804214


In [11]:
print(feature_vector.shape)
feature_vector = feature_vector.to('cpu').detach().numpy()
feature_vector = np.mean(feature_vector, axis=0)
print(feature_vector.shape)

torch.Size([10, 2468, 1024])
(2468, 1024)


In [12]:
import datetime
feature_vector_df = pd.DataFrame(data=feature_vector)

method_name = 'seed0/Feature_vector_base' #何の結果か、どのような加工を加えているか、何個ずつか
prefix = 'average' #iteration=10の平均を取っているのでaverageというprefixをつけている。
ext = ".csv"
df = feature_vector_df
#-------------------------------------
save_dir = FILE_DIR / Path(method_name) 
# print(save_dir)
save_dir.mkdir(exist_ok=True, parents=True)
timestr = str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M'))
# print(timestr)
file_name = prefix + timestr + ext
df.to_csv(save_dir/file_name, index=False)
print("Saved in:", save_dir/file_name)
# slack_notify("Saved in:"+ str(save_dir/file_name))

Saved in: ../Pointnet_Pointnet2_pytorch/../Mywork/Results/Files/seed0/Feature_vector_base/average2022-02-24_20-15.csv
